# Grab carbon boxes

This is likely to require the restarts to work with multi-gas - so needs fair version >= 1.6.3 and < 2.0

## import required packages

In [ ]:
import fair
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import json
from tqdm import tqdm
import pickle
from climateforcing.utils import mkdir_p

## check version installed

In [ ]:
fair.__version__

## get SSP2-4.5 emissions pathways (original CMIP6 version)

In [ ]:
ssp_df = pd.read_csv('../data_input/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
years = np.arange(1750, 2021)

startyear = 1750
first_scenyear = 2015
last_scenyear = 2020
first_row = int(first_scenyear-startyear)
last_row = int(last_scenyear-startyear)

species = [
    '|CO2|MAGICC Fossil and Industrial',
    '|CO2|MAGICC AFOLU',
    '|CH4',
    '|N2O',
    '|Sulfur',
    '|CO',
    '|VOC',
    '|NOx',
    '|BC',
    '|OC',
    '|NH3',
    '|CF4',
    '|C2F6',
    '|C6F14',
    '|HFC23',
    '|HFC32',
    '|HFC4310mee',
    '|HFC125',
    '|HFC134a',
    '|HFC143a',
    '|HFC227ea',
    '|HFC245fa',
    '|SF6',
    '|CFC11',
    '|CFC12',
    '|CFC113',
    '|CFC114',
    '|CFC115',
    '|CCl4',
    '|CH3CCl3',
    '|HCFC22',
    '|HCFC141b',
    '|HCFC142b',
    '|Halon1211',
    '|Halon1202',
    '|Halon1301',
    '|Halon2402',
    '|CH3Br',
    '|CH3Cl',
]

unit_convert = np.ones(40)
unit_convert[1] = 12/44/1000
unit_convert[2] = 12/44/1000
unit_convert[4] = 28/44/1000
unit_convert[5] = 32/64
unit_convert[8] = 14/46

emissions_out = np.ones((271, 40)) * np.nan
emissions_out[:,0] = years

years_future = [2015, 2020]
for i, specie in enumerate(species):
    emissions_out[:first_row,i+1] = ssp_df.loc[
        (ssp_df['Model']=='MESSAGE-GLOBIOM')&
        (ssp_df['Region']=='World')&
        (ssp_df['Scenario']=='ssp245')&
        (ssp_df['Variable'].str.endswith(specie)),str(startyear):'2014']*unit_convert[i+1]
    f = interp1d(years_future, ssp_df.loc[
        (ssp_df['Model']=='MESSAGE-GLOBIOM')&
        (ssp_df['Region']=='World')&
        (ssp_df['Scenario']=='ssp245')&
        (ssp_df['Variable'].str.endswith(specie)), '2015':'2020'].dropna(axis=1))
    emissions_out[first_row:(last_row+1), i+1] = f(
        np.arange(first_scenyear, last_scenyear+1)
    )*unit_convert[i+1]

In [ ]:
emissions_out

## get FaIR v1.6.2 AR6 config

It's ok to run 1.6.4 with the 1.6.2 config, they are the same model but the slightly later version enables multigas restarts

In [ ]:
with open('../data_input/fair-1.6.2/fair-1.6.2-wg3-params.json') as f:
    config_list = json.load(f)

## define convenience functions for running FaIR

The main differences to here and AR6 are:
- we specify `restart_out=True`
- and we want to truncate the runs at 2020, so we have to chop the natural emissions time series early
- we have to run impulse response rather than energy balance

### convert energy balance to impulse response

See:
- https://github.com/chrisroadmap/ar6/blob/main/notebooks/140_WG3_draw_fair_samples.ipynb
- https://github.com/chrisroadmap/ar6/blob/main/notebooks/200_chapter6_impulse_response_function.ipynb

In [ ]:
def _calculate_geoffroy_helper_parameters(
    cmix, cdeep, lambda0, efficacy, eta, f2x
):

    ecs = f2x/lambda0
    tcr = f2x/(lambda0 + efficacy*eta)
    
    b_pt1 = (lambda0 + efficacy * eta) / cmix
    b_pt2 = eta / cdeep
    b = b_pt1 + b_pt2
    b_star = b_pt1 - b_pt2
    delta = b ** 2 - (4 * lambda0 * eta) / (cmix * cdeep)

    taucoeff = cmix * cdeep / (2 * lambda0 * eta)
    d1 = taucoeff * (b - delta ** 0.5)
    d2 = taucoeff * (b + delta ** 0.5)

    phicoeff = cmix / (2 * efficacy * eta)
    phi1 = phicoeff * (b_star - delta ** 0.5)
    phi2 = phicoeff * (b_star + delta ** 0.5)

    qdenom = cmix * (phi2 - phi1)
    q1 = d1 * phi2 / qdenom
    q2 = -d2 * phi1 / qdenom

    out = {
        "d1": d1,
        "d2": d2,
        "q1": q1,
        "q2": q2,
        "ecs": ecs,
        "tcr": tcr,
    }
    return out

In [ ]:
def run_fair(args):
    c, f, t, restart_out = fair.forward.fair_scm(**args)
    return (
        c[:,0],
        np.sum(f, axis=1), 
        t-np.mean(t[100:151]),
        restart_out[0][0],
        restart_out[0][1], 
        restart_out[0][2], 
        restart_out[0][3])

def fair_process(emissions):
    updated_config = []
    for i, cfg in enumerate(config_list):
        updated_config.append({})
        for key, value in cfg.items():
            if isinstance(value, list):
                updated_config[i][key] = np.asarray(value)
            else:
                updated_config[i][key] = value
        updated_config[i]['emissions'] = emissions
        updated_config[i]['diagnostics'] = 'AR6'
        updated_config[i]["efficacy"] = np.ones(45)
        updated_config[i]["gir_carbon_cycle"] = True
        updated_config[i]["temperature_function"] = "Millar"
        updated_config[i]["aerosol_forcing"] = "aerocom+ghan2"
        updated_config[i]["fixPre1850RCP"] = False
        updated_config[i]["restart_out"] = True
        solar_truncated = updated_config[i]["F_solar"][:271]
        updated_config[i]['F_solar'] = solar_truncated
        volcanic_truncated = updated_config[i]["F_volcanic"][:271]
        updated_config[i]['F_volcanic'] = volcanic_truncated
        nat_truncated = updated_config[i]['natural'][:271, :]
        updated_config[i]['natural'] = nat_truncated
        out = _calculate_geoffroy_helper_parameters(
            cfg['ocean_heat_capacity'][0],
            cfg['ocean_heat_capacity'][1],
            cfg['lambda_global'],
            cfg['deep_ocean_efficacy'],
            cfg['ocean_heat_exchange'],
            cfg['F2x']
        )
        updated_config[i]['d'] = np.array([out['d1'], out['d2']])
        updated_config[i]['q'] = np.array([out['q1'], out['q2']])
        
    shape = (271, len(updated_config))
    c_co2 = np.ones(shape) * np.nan
    t = np.ones(shape) * np.nan
    f_tot = np.ones(shape) * np.nan
    cbox = np.ones((4, len(updated_config))) * np.nan
    
    for i, cfg in tqdm(enumerate(updated_config), total=len(updated_config), position=0, leave=True):
        c_co2[:,i], f_tot[:,i], t[:,i], cbox[0,i], cbox[1,i], cbox[2,i], cbox[3,i] = run_fair(updated_config[i])


    return c_co2, f_tot, t, cbox

## Run and print results

In [ ]:
results_out = {}
(
    results_out['concentrations'],
    results_out['effective_radiative_forcings'],
    results_out['temperatures'],
    results_out['carbon_boxes']
) = fair_process(emissions_out)

In [ ]:
results_out['carbon_boxes'][0,:]

In [ ]:
results_out['carbon_boxes'][1,:]

In [ ]:
results_out['carbon_boxes'][2,:]

In [ ]:
results_out['carbon_boxes'][3,:]

In [ ]:
print(results_out['carbon_boxes'][0,:].mean(), results_out['carbon_boxes'][0,:].std())

In [ ]:
print(results_out['carbon_boxes'][1,:].mean(), results_out['carbon_boxes'][1,:].std())

In [ ]:
print(results_out['carbon_boxes'][2,:].mean(), results_out['carbon_boxes'][2,:].std())

In [ ]:
print(results_out['carbon_boxes'][3,:].mean(), results_out['carbon_boxes'][3,:].std())

In [ ]:
results_out

## Save results as a pickle object

In [ ]:
mkdir_p('../data_output/fair-1.6.4/')
with open('../data_output/fair-1.6.4/fair-1.6.4-carbon-boxes.pkl', 'wb') as f:
    pickle.dump(results_out, f)